<a href="https://colab.research.google.com/github/MLo7Ghinsan/DiffSinger_colab_notebook_MLo7/blob/main/DDSP_Vocoder_training_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title # Mount Google Drive and Setup


#ill put the imports here too ig
from IPython.display import clear_output
import os
import shutil
import random
import yaml
from google.colab import drive
drive.mount("/content/drive")
!rm -rf /content/sample_data

!git clone https://github.com/yxlllc/pc-ddsp
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -r pc-ddsp/requirements.txt
!pip install pydub
from pydub import AudioSegment



In [ ]:
#@title # Extract Data & Preprocessing
#@markdown ___

#@markdown Path to zip file containing your audio data
data_zip_path = "" # @param {type:"string"}

#@markdown Name of the folder that stuff will be saved in (or just think of speaker's name)
folder_name = "" # @param {type:"string"}

#@markdown Config type you want to use
train_config = "combsub" # @param ["combsub", "sins"]

#@markdown Pitch extractor algorithm
f0_ext = "parselmouth" # @param ["parselmouth", "dio", "harvest"]
f0_min = 12 #C0
f0_max = 2100 #about C7

#@markdown Model save interval
interval_val = 2000 # @param {type:"slider", min:100, max:10000, step:100}

#@markdown Model save path
expdir = "" # @param {type:"string"}

train_path = f"{expdir}/{folder_name}/audio_data/train/audio"
val_path = f"{expdir}/{folder_name}/audio_data/val/audio"

train_path_conf = f"{expdir}/{folder_name}/audio_data/train"
val_path_conf = f"{expdir}/{folder_name}/audio_data/val"

expdir_name = expdir + "/" + folder_name

!rm -rf {expdir}/{folder_name}/* >/dev/null 2>&1

if not os.path.exists(train_path):
    os.makedirs(train_path)
    os.makedirs(val_path)
!7z e "$data_zip_path" -o{train_path} *.wav -r

def get_wav_duration(wav_file):
    audio = AudioSegment.from_file(wav_file)
    return len(audio) / 1000

wav_files = [f for f in os.listdir(train_path) if f.endswith(".wav")]

if not wav_files: #incase there's no wav in there which i doubt that there wont be any
    print("No WAV files found in the folder.")
else:
    shortest_duration = float('inf')
    for wav_file in wav_files:
        duration = get_wav_duration(os.path.join(train_path, wav_file))
        if duration < shortest_duration:
            shortest_duration = duration
    duration = min(2, shortest_duration) #use this for the config later

#@markdown Amount of validation files you want to use (can't exceed the amount of train files)
val_amount = 8 # @param {type:"slider", min:1, max:18, step:1}

all_shits = os.listdir(train_path)
random.shuffle(all_shits)

val_files = all_shits[:val_amount]

for rats in val_files:
    og = os.path.join(train_path, rats)
    dest = os.path.join(val_path, rats)
    shutil.copy2(og, dest)
print("\n")
print(f"using {val_files} as validation files")

with open(f"/content/pc-ddsp/configs/{train_config}.yaml", "r") as config:
    ew = yaml.safe_load(config)
ew["data"]["f0_extractor"] = f0_ext
ew["data"]["f0_min"] = f0_min
ew["data"]["f0_max"] = f0_max
ew["data"]["duration"] = duration
ew["data"]["train_path"] = train_path_conf
ew["data"]["valid_path"] = val_path_conf
ew["train"]["interval_val"] = interval_val
ew["env"]["expdir"] = expdir_name
with open(f"/content/pc-ddsp/configs/{train_config}.yaml", "w") as config:
    yaml.dump(ew, config)
print("\n")

!python /content/pc-ddsp/preprocess.py -c /content/pc-ddsp/configs/{train_config}.yaml

In [ ]:
#@title #Tensorboard
#@markdown ___

#@markdown path to your save dir basically
logdir = "" # @param {type:"string"}
!tensorboard --logdir={logdir}

In [ ]:
#@title #Training
#@markdown ___

#@markdown Only check this if you want to resume training
resume_training = False # @param {type:"boolean"}

#@markdown Path to the config you want to resume
re_config = "" # @param {type:"string"}

if resume_training:
    !python /content/pc-ddsp/train.py -c {re_config}
else:
    !python /content/pc-ddsp/train.py -c /content/pc-ddsp/configs/{train_config}.yaml

In [ ]:
#@title #Export your model
#@markdown ___
#@markdown Path to your vocoder model

pt_path = "" # @param {type:"string"}

!python /content/pc-ddsp/export.py -m {pt_path} --traced